In [26]:
import sys
import math
import pickle
import numpy as np

k = 2
sys.setrecursionlimit(1500)

In [27]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [28]:
digitImage = inputDigit()
print(len(digitImage))
print(len(digitImage[139999]))

140000
28


In [29]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [30]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000
[('0', 479), ('1', 563), ('2', 488), ('3', 493), ('4', 535), ('5', 434), ('6', 501), ('7', 550), ('8', 462), ('9', 495)]


In [31]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training0=np.zeros(shape=(10, 28, 28))
training1=np.zeros(shape=(10, 28, 28))

count = 0
for i in range(5000):
    for row in range(28):
        for col in range(28):
            if digitImage[i*28 + row][col] == ' ':
                training1[int(label[i])][row][col] += 0
                training0[int(label[i])][row][col] += 1
            else:
                training1[int(label[i])][row][col] += 1
                training0[int(label[i])][row][col] += 0

for i in range(10):
    training1[i] = (training1[i] + k) / (class_[i][1] + k * 2)
    training0[i] = (training0[i] + k) / (class_[i][1] + k * 2)
print(training0[0][0])
print(training1[0][0])



[ 0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921]
[ 0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079]


In [ ]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")

answer = np.zeros(1000)
for i in range(1000):
    test_image = np.zeros(shape=(28,28))
    for row in range(28):
        for col in range(28):
            if test_rough[i*28+row][col] == ' ':
                test_image[row][col] = 0
            else:
                test_image[row][col] = 1
             
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(28):
            for col in range(28):
                if test_image[row][col] == 0:
                    posteriori[class_num] += math.log(training0[class_num][row][col])
                else:
                    posteriori[class_num] += math.log(training1[class_num][row][col] )   
    answer[i] = np.argmax(posteriori)
    
print(answer)



In [33]:
testlabels = readRough("digitdata/testlabels")


In [34]:
def confusion_matrix():
    num_each_class = np.zeros(10)
    conf_matrix = np.zeros(shape=(10,10))
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [35]:
conf_matrix = confusion_matrix()
conf_matrix = np.around(conf_matrix, 2)
for row in conf_matrix:
    print(row)
diag_sum = 0
for i in range(10):
    diag_sum += conf_matrix[i][i]
print(diag_sum)

[ 0.78  0.    0.    0.01  0.    0.01  0.13  0.    0.03  0.03]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.02  0.57  0.25  0.03  0.01  0.    0.07  0.02  0.02  0.01]
[ 0.    0.42  0.    0.49  0.    0.    0.01  0.06  0.01  0.01]
[ 0.    0.18  0.    0.    0.57  0.    0.03  0.    0.    0.22]
[ 0.03  0.34  0.    0.22  0.04  0.11  0.04  0.03  0.03  0.15]
[ 0.    0.31  0.    0.    0.07  0.    0.63  0.    0.    0.  ]
[ 0.    0.32  0.    0.    0.02  0.    0.    0.57  0.01  0.08]
[ 0.02  0.46  0.    0.06  0.01  0.    0.04  0.02  0.25  0.15]
[ 0.01  0.11  0.    0.01  0.04  0.    0.    0.01  0.01  0.81]
5.46
